In [1]:
from smolagents.local_python_executor import LocalPythonExecutor

custom_executor = LocalPythonExecutor(["numpy"])

In [ ]:
def run_capture_exception(command: str):
    try:
        custom_executor(harmful_command) # type: ignore
    except Exception as e:
        print("ERROR:\n", e)

In [3]:
# Example 1: non-defined command
# In Jupyter it works
!echo Bad command

Bad command


In [ ]:
[
    "re",
    "queue",
    "random",
    "statistics",
    "unicodedata",
    "itertools",
    "math",
    "stat",
    "time",
    "datetime",
    "collections",
    "numpy",
]

['re',
 'queue',
 'random',
 'statistics',
 'unicodedata',
 'itertools',
 'math',
 'stat',
 'time',
 'datetime',
 'collections',
 'numpy',
 'os']

### RUNNING IN SANDBOXES

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

E2B_API_KEY = os.getenv("E2B_API_KEY")

In [3]:
from smolagents import CodeAgent, HfApiModel, Tool

model = HfApiModel()

class VisitWebpageTool(Tool):
    name = "visit_webpage"
    description = "Visits a webpage at the given url and reads its content as a markdown string. Use this to browse webpages."
    inputs = {
        "url": {
            "type": "string",
            "description": "The url of the webpage to visit.",
        }
    }
    output_type = "string"

    def __init__(self, max_output_length: int = 40000):
        super().__init__()
        self.max_output_length = max_output_length

    def forward(self, url: str) -> str:
        try:
            import re
            import requests
            from markdownify import markdownify
            from requests.exceptions import RequestException
            from smolagents.utils import truncate_content
        except ImportError as e:
            raise ImportError(
                "You must install packages `markdownify` and `requests` to run this tool: for instance run `pip install markdownify requests`."
            ) from e
        try:
            response = requests.get(url, timeout=20)
            response.raise_for_status()
            markdown_content = markdownify(response.text).strip()
            markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
            return truncate_content(markdown_content, self.max_output_length)
        except requests.exceptions.Timeout:
            return "The request timed out. Please try again later or check the URL."
        except RequestException as e:
            return f"Error fetching the webpage: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"


agent = CodeAgent(
    tools=[VisitWebpageTool()],
    model=model,
    executor_type="e2b",
    executor_kwargs={"api_key": E2B_API_KEY},
    max_steps=5,
)

Initializing executor, hold on...

E2B is running

In [ ]:
output = agent.run("Give me one of the top github repos from organization huggingface.")
print("E2B executor result:", output)